## This Notebook is our PlayGround

### Module Imports

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
import os
from dotenv import load_dotenv

import base64

from github import Github,Auth
import google.generativeai as genai

In [17]:
load_dotenv()

True

In [18]:
def create_and_get_gemini_model():
    genai.configure(api_key=os.getenv('GOOGLE_GEMINI_API_KEY'))
    model = genai.GenerativeModel('gemini-pro')
    return model

In [19]:
def create_get_github_object():
    g=Github(auth=Auth.Token(os.getenv('GITHUB_ACCESS_TOKEN')))
    return g

In [20]:
def get_username_and_repo_from_url(url):
    starting='https://github.com/'
    return url[len(starting):]

In [21]:
def get_repository_object(github_object,repo_url):
    uname_and_reponame = get_username_and_repo_from_url(repo_url)
    repo_object = github_object.get_repo(uname_and_reponame)
    return repo_object

In [22]:
def load_prompt(file_path='./prompt.txt'):
    with open(file_path,'r') as f:
        prompt=f.read()
    return prompt

In [52]:
def get_report_from_gemini(model,file_content):
    prompt=load_prompt()
    report = model.generate_content(f"""{prompt}/n{file_content}""")
    buffer=""""""
    for part in report.parts:
        buffer+=part.text
    return buffer

In [24]:
def decode_bytes_to_str(byte_data):
    decoded = base64.b64decode(byte_data)
    return decoded

In [25]:
dirs_to_ignore=['node_modules','.github']
files_to_ignore=['LICENSE','.gitignore','.git','.gitattributes','.env','.bat','.sh']
extensions_to_ignore=['md','txt','pptx','docx','xlsx','csv','db','sqlite','yaml','abi','bin','json','onnx','jpg','png','jpeg']

In [50]:
def get_file_content_from_repo(repo_object,file_name):
    encoded_file_content = repo_object.get_contents(file_name).content
    decoded_file_content = decode_bytes_to_str(encoded_file_content)
    return decoded_file_content

In [57]:
def write_file(uname_and_reponame,file_path,file_content):
    with open(f"./reports_generated/{uname_and_reponame}/{file_path}",'w',encoding='utf-8') as f:
        f.write(file_content)

In [36]:
def convert_to_md(file_name):
    return f"{file_name.replace(".","_")}_report.md"

In [38]:
def store_report_in_local_directory(uname_and_reponame,repo_object,gemini_model):
    contents= repo_object.get_contents("")
    while contents:
        item = contents.pop(0)
        if (item.type == "dir" and item.name not in dirs_to_ignore):
            if not os.path.exists(f"./reports_generated/{uname_and_reponame}/{item.path}"):
                os.mkdir(f"./reports_generated/{uname_and_reponame}/{item.path}")
            contents.extend(repo_object.get_contents(item.path))
        elif (item.type=="file" and item.name not in files_to_ignore and item.name.split(".")[-1] not in extensions_to_ignore):
            file_content = get_file_content_from_repo(repo_object,item.path)
            generated_report = get_report_from_gemini(gemini_model,file_content)
            write_file(uname_and_reponame,convert_to_md(item.path),generated_report)

In [48]:
github_object   = create_get_github_object()
repo_url        = "https://github.com/Maran1947/Stockify"
repo_object     = get_repository_object(github_object,repo_url)
gemini_model   = create_and_get_gemini_model()

In [41]:
uname_and_reponame = get_username_and_repo_from_url(repo_url)

In [58]:
os.mkdir(f"./reports_generated/{uname_and_reponame.split("/")[0]}")
os.mkdir(f"./reports_generated/{uname_and_reponame.split("/")[0]}/{uname_and_reponame.split("/")[1]}")

In [59]:
store_report_in_local_directory(uname_and_reponame,repo_object,gemini_model)